In [1]:
import numpy as np
import os
import random as r
import matplotlib.pyplot as plt
import tensorflow as tf 
import test_multiprocess as tm

In [2]:
files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data'
images_dir = '221123_mtStayGold_both_ZEISS_fl_pos'
images_neg_dir = '221123_mtStayGold_both_ZEISS_fl_neg'
import multiprocessing
sigmas = [2,3,4,5,6]
number_of_augmentations = 40
box_size=15
offset=3
background=0.1

for sigma in sigmas:
    print('sigma =',sigma)
    p = multiprocessing.Process(target=tm.load_aug_train, args=(files_dir, images_dir, images_neg_dir, sigma, number_of_augmentations,box_size, offset, background)) 
    p.start() 
    p.join()


In [23]:
import multiprocessing


sigmas = [1,2,3,4,5,6,7]
number_of_augmentations = 30

for sigma in sigmas:
    print('sigma =',sigma)
    p = multiprocessing.Process(target=tm.load_aug_train, args=(files_dir, images_dir, images_neg_dir, sigma, number_of_augmentations)) 
    p.start() 
    p.join()


0 AA
1 BB
2 CC


In [33]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from mitosplit_net import util
from training_functions import create_model,train_model
from albumentations import Compose, Rotate, RandomRotate90, HorizontalFlip, Flip, VerticalFlip
import os
from os import path
import random as r
import imageio
from import_augmentation_function import import_fun, aug_fun, import_fun_neg, normalization_fun, normalization_fun_g, import_aug_fun
import tensorflow as tf 
from test_multiprocess import zero_frames

files_dir = r'C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1_4.tiff'
files_dir_g= r'C:\Users\roumba\Documents\Software\deep-events\training_data\220915_mtstaygold_cos7_ZEISS_fl_pos\image_220915_cos7_GFP_1_sigma5_4gauss.tiff'

data_ratio= 0.1
data_split_state = None

img_gauss = Image.open(files_dir_g)
image_array_gauss = np.zeros((img_gauss.n_frames,256,256))
for i in range(0, img_gauss.n_frames):
    img_gauss.seek(i)
    image_array_gauss[i,:,:] = np.array(img_gauss)
all_image_array_gauss = image_array_gauss

img = Image.open(files_dir)
image_array = np.zeros((img.n_frames,256,256))
for i in range(0,img.n_frames):
    img.seek(i)
    image_array[i,:,:] = np.array(img)
all_image_array = image_array
print(all_image_array.shape)
print(all_image_array_gauss.shape)

(15, 256, 256)
(15, 256, 256)


In [34]:
                        ## NORMALIZATION ##

norm_image_array= normalization_fun(all_image_array, 0.1)
norm_image_array_gauss = normalization_fun_g(all_image_array_gauss, 0.1)
augmentation_data, data_val, augmentation_data_gauss, data_gauss_val =  train_test_split(norm_image_array, norm_image_array_gauss,
                                                                                                       test_size=data_ratio, random_state=data_split_state)
print(augmentation_data.shape)
print(augmentation_data_gauss.shape)

data_aug = np.zeros((np.size(augmentation_data , 0),128,128))
data_gauss_aug = np.zeros((np.size(augmentation_data , 0),128,128))

print(data_aug.shape)
print(data_gauss_aug.shape)

for frame_index in range(np.size(data_aug , 0)):
        data_aug[frame_index] = augmentation_data[frame_index, 64:192 , 64:192]
        data_gauss_aug[frame_index] = augmentation_data_gauss[frame_index, 64:192 , 64:192]
print(data_aug.shape)
print(data_gauss_aug.shape) 
print(data_aug)     

(13, 256, 256)
(13, 256, 256)
(13, 128, 128)
(13, 128, 128)
(13, 128, 128)
(13, 128, 128)
[[[1.43689123e-02 1.25547841e-02 1.53861804e-02 ... 4.89077083e-01
   6.34682061e-01 7.28050321e-01]
  [1.92348448e-02 2.03707942e-02 2.00317049e-02 ... 6.20457262e-01
   7.72521892e-01 8.23520935e-01]
  [2.21171045e-02 2.63557217e-02 3.04926121e-02 ... 7.67215144e-01
   8.71552945e-01 8.61448081e-01]
  ...
  [1.00751931e-01 8.51368649e-02 7.59475429e-02 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [1.31100430e-01 1.07245492e-01 8.70188110e-02 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [1.79081576e-01 1.41730882e-01 1.04159779e-01 ... 0.00000000e+00
   0.00000000e+00 0.00000000e+00]]

 [[1.50470910e-02 1.67086290e-02 1.95061163e-02 ... 9.15583700e-01
   9.95286658e-01 9.61106449e-01]
  [1.99130236e-02 2.24392394e-02 2.76273069e-02 ... 9.28486051e-01
   9.39286047e-01 8.86269423e-01]
  [2.90175733e-02 2.90853912e-02 3.60028144e-02 ... 8.34795656e-01
   7.82270712e-01 6.8423997

In [35]:
number_of_augmentations=5

for j in range(number_of_augmentations):
    transform = Compose([Rotate(limit=45, p=0.5), RandomRotate90(p=0.5), HorizontalFlip(p=0.5), Flip(p=0.5), VerticalFlip(p=0.5)])
    print('augmentation', j)
    augment_data, augment_data_gauss = aug_fun(augmentation_data, augmentation_data_gauss,transform)
    # for frame_index in range(np.size(augment_data , 0)):
    #     augment_data= augment_data[frame_index, 64:192 , 64:192]
    #     augment_data_gauss= augment_data_gauss[frame_index, 64:192 , 64:192]
    data_aug = np.concatenate((data_aug, augment_data[:, 64:192 , 64:192]))
    print(data_aug.shape)
    data_gauss_aug = np.concatenate((data_gauss_aug,augment_data_gauss[:, 64:192 , 64:192]))

augmentation 0


100%|██████████| 13/13 [00:00<00:00, 406.11it/s]


(26, 128, 128)
augmentation 1


100%|██████████| 13/13 [00:00<00:00, 566.36it/s]


(39, 128, 128)
augmentation 2


...

(52, 128, 128)
augmentation 3


100%|██████████| 13/13 [00:00<00:00, 649.62it/s]


(65, 128, 128)
augmentation 4


100%|██████████| 13/13 [00:00<00:00, 721.55it/s]

(78, 128, 128)
